In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Epoch  1, Batch   1 - Loss: 68173.3047 Validation Accuracy: 0.054688
Epoch  1, Batch   2 - Loss: 48835.0586 Validation Accuracy: 0.066406
Epoch  1, Batch   3 - Loss: 43632.3516 Validation Accuracy: 0.085938
Epoch  1, Batch   4 - Loss: 38653.3828 Validation Accuracy: 0.085938
Epoch  1, Batch   5 - Loss: 34165.7109 Validation Accuracy: 0.113281
Epoch  1, Batch   6 - Loss: 33732.8047 Validation Accuracy: 0.113281
Epoch  1, Batch   7 - Loss: 28963.7188 Validation Accuracy: 0.117188
Epoch  1, Batch   8 - Loss: 28433.6875 Validation Accuracy: 0.132812
Epoch  1,

Epoch  1, Batch 114 - Loss:  6932.4731 Validation Accuracy: 0.625000
Epoch  1, Batch 115 - Loss:  4787.9224 Validation Accuracy: 0.613281
Epoch  1, Batch 116 - Loss:  4290.3848 Validation Accuracy: 0.617188
Epoch  1, Batch 117 - Loss:  4118.5527 Validation Accuracy: 0.609375
Epoch  1, Batch 118 - Loss:  5395.6748 Validation Accuracy: 0.613281
Epoch  1, Batch 119 - Loss:  3801.6831 Validation Accuracy: 0.628906
Epoch  1, Batch 120 - Loss:  4695.3936 Validation Accuracy: 0.621094
Epoch  1, Batch 121 - Loss:  3890.4346 Validation Accuracy: 0.613281
Epoch  1, Batch 122 - Loss:  5158.1333 Validation Accuracy: 0.605469
Epoch  1, Batch 123 - Loss:  4591.9048 Validation Accuracy: 0.613281
Epoch  1, Batch 124 - Loss:  5684.2085 Validation Accuracy: 0.613281
Epoch  1, Batch 125 - Loss:  4213.8486 Validation Accuracy: 0.613281
Epoch  1, Batch 126 - Loss:  4622.0029 Validation Accuracy: 0.625000
Epoch  1, Batch 127 - Loss:  3471.3530 Validation Accuracy: 0.628906
Epoch  1, Batch 128 - Loss:  3061.

Epoch  1, Batch 233 - Loss:  1921.5969 Validation Accuracy: 0.722656
Epoch  1, Batch 234 - Loss:  2936.2705 Validation Accuracy: 0.722656
Epoch  1, Batch 235 - Loss:  1901.2753 Validation Accuracy: 0.714844
Epoch  1, Batch 236 - Loss:  2610.2385 Validation Accuracy: 0.718750
Epoch  1, Batch 237 - Loss:  2146.6265 Validation Accuracy: 0.718750
Epoch  1, Batch 238 - Loss:  2947.3274 Validation Accuracy: 0.726562
Epoch  1, Batch 239 - Loss:  2530.4136 Validation Accuracy: 0.726562
Epoch  1, Batch 240 - Loss:  2063.1152 Validation Accuracy: 0.718750
Epoch  1, Batch 241 - Loss:  2611.6753 Validation Accuracy: 0.730469
Epoch  1, Batch 242 - Loss:  3222.6289 Validation Accuracy: 0.718750
Epoch  1, Batch 243 - Loss:  2624.4180 Validation Accuracy: 0.714844
Epoch  1, Batch 244 - Loss:  2413.9875 Validation Accuracy: 0.718750
Epoch  1, Batch 245 - Loss:  2340.8132 Validation Accuracy: 0.714844
Epoch  1, Batch 246 - Loss:  2704.6743 Validation Accuracy: 0.722656
Epoch  1, Batch 247 - Loss:  2407.

Epoch  1, Batch 352 - Loss:  1644.8083 Validation Accuracy: 0.769531
Epoch  1, Batch 353 - Loss:  1476.3301 Validation Accuracy: 0.769531
Epoch  1, Batch 354 - Loss:  1562.2766 Validation Accuracy: 0.769531
Epoch  1, Batch 355 - Loss:  1899.4109 Validation Accuracy: 0.765625
Epoch  1, Batch 356 - Loss:  2380.3765 Validation Accuracy: 0.765625
Epoch  1, Batch 357 - Loss:  1995.3445 Validation Accuracy: 0.769531
Epoch  1, Batch 358 - Loss:  2211.8296 Validation Accuracy: 0.757812
Epoch  1, Batch 359 - Loss:  1885.8013 Validation Accuracy: 0.773438
Epoch  1, Batch 360 - Loss:  1586.7206 Validation Accuracy: 0.769531
Epoch  1, Batch 361 - Loss:  1713.7292 Validation Accuracy: 0.769531
Epoch  1, Batch 362 - Loss:  2388.8621 Validation Accuracy: 0.777344
Epoch  1, Batch 363 - Loss:  1759.1169 Validation Accuracy: 0.785156
Epoch  1, Batch 364 - Loss:  1663.5371 Validation Accuracy: 0.765625
Epoch  1, Batch 365 - Loss:  1900.6250 Validation Accuracy: 0.765625
Epoch  1, Batch 366 - Loss:  1117.

Epoch  2, Batch  42 - Loss:  1692.0132 Validation Accuracy: 0.800781
Epoch  2, Batch  43 - Loss:  1264.8588 Validation Accuracy: 0.808594
Epoch  2, Batch  44 - Loss:  1687.8074 Validation Accuracy: 0.808594
Epoch  2, Batch  45 - Loss:   723.4609 Validation Accuracy: 0.804688
Epoch  2, Batch  46 - Loss:  1128.1637 Validation Accuracy: 0.800781
Epoch  2, Batch  47 - Loss:  1475.7910 Validation Accuracy: 0.796875
Epoch  2, Batch  48 - Loss:  2047.9138 Validation Accuracy: 0.796875
Epoch  2, Batch  49 - Loss:  1132.2572 Validation Accuracy: 0.796875
Epoch  2, Batch  50 - Loss:  1255.0276 Validation Accuracy: 0.792969
Epoch  2, Batch  51 - Loss:  1665.0210 Validation Accuracy: 0.800781
Epoch  2, Batch  52 - Loss:  1118.7972 Validation Accuracy: 0.796875
Epoch  2, Batch  53 - Loss:  1030.5846 Validation Accuracy: 0.792969
Epoch  2, Batch  54 - Loss:  1947.7341 Validation Accuracy: 0.789062
Epoch  2, Batch  55 - Loss:  1198.6479 Validation Accuracy: 0.789062
Epoch  2, Batch  56 - Loss:  1050.

KeyboardInterrupt: 

In [2]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def conv2d(input):
    # Filter (weights and bias)
    F_W = tf.Variable(tf.truncated_normal((2, 2, 1, 3)))
    F_b = tf.Variable(tf.zeros(3))
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b



In [ ]:
"""
Set the values to `strides` and `ksize` such that
the output shape after pooling is (1, 2, 2, 1).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.max_pool` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    # TODO: Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
    ksize = [1, 2, 2, 1]
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'VALID'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#max_pool
    return tf.nn.max_pool(input, ksize, strides, padding)
    
out = maxpool(X)